# Gegelati Code Generation

Declare covered Games and platforms

In [ ]:
#global platforms = ["xeon", "xeonSwitch", "laptop", "laptopSwitch", "jetson", "jetsonSwitch", "rpi2", "rpi2Switch"];
global platforms = ["xeon", "laptop", "jetson", "rpi2"];
#global platforms = ["xeon", "xeonStack", "laptop", "laptopStack", "jetson", "jetsonStack", "rpi2" , "rpi2Stack"];
#global platforms = ["xeon", "xeonStack"];
#global platforms = ["xeonStack", "laptopStack", "jetsonStack", "rpi2Stack"];

global games = ["alien", "asteroids", "centipede", "fishing_derby", "frostbite"];

global seeds = 0:9

global nbRunPerConfig = size(seeds)[1]

## Load the database from CSV files.

In [ ]:
include("results.jl")

In [ ]:
# Load game stats
(dbCodegen, dbTPG) = loadDatabaseFromCsv();
println("Platforms: $(platforms)")
println("Games: $(games)")
println("Number of entries in the database (should be $(size(seeds)[1]) * nb_games * nb_platforms): $(size(dbCodegen)[1])")


## TPG Stats

In [ ]:

# Load graph stats
dbTPGStats = loadTPGStatsDatabase();

tpgStats = DataFrame(game=String[], nbTeams=Float64[], nbPrograms=Float64[], linePerProg=Float64[], nbActionPerGame=Float64[], nbTeamExecPerProg=Float64[], nbProgExecPerProg=Float64[])

for game in games
    entry = [
        game,
        mean(dbTPGStats[dbTPGStats.game.==game, :nbTeams]),
        mean(dbTPGStats[dbTPGStats.game.==game, :nbPrograms]),
        mean(dbTPGStats[dbTPGStats.game.==game, :linePerProgram]),
        mean(dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions]),
        mean(dbTPGStats[dbTPGStats.game.==game, :nbTeamExec] ./ dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions]),
        mean(dbTPGStats[dbTPGStats.game.==game, :nbProgExec] ./ dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions])
    ]
    push!(tpgStats, entry)
end

tpgVar = DataFrame(game=String[], nbTeams=Float64[], nbPrograms=Float64[], linePerProg=Float64[], nbActionPerGame=Float64[], nbTeamExecPerAction=Float64[], nbProgExecPerAction=Float64[])

for game in games
    entry = [
        game,
        100 * std(dbTPGStats[dbTPGStats.game.==game, :nbTeams]) / mean(dbTPGStats[dbTPGStats.game.==game, :nbTeams]),
        100 * std(dbTPGStats[dbTPGStats.game.==game, :nbPrograms]) / mean(dbTPGStats[dbTPGStats.game.==game, :nbPrograms]),
        100 * std(dbTPGStats[dbTPGStats.game.==game, :linePerProgram]) / mean(dbTPGStats[dbTPGStats.game.==game, :linePerProgram]),
        100 * std(dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions]) / mean(dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions]),
        100 * std(dbTPGStats[dbTPGStats.game.==game, :nbTeamExec] ./ dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions]) / mean(dbTPGStats[dbTPGStats.game.==game, :nbTeamExec] ./ dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions]),
        100 * std(dbTPGStats[dbTPGStats.game.==game, :nbProgExec] ./ dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions]) / mean(dbTPGStats[dbTPGStats.game.==game, :nbProgExec] ./ dbCodegen[(dbCodegen.platform.==platforms[1]).&(dbCodegen.game.==game), :actions])
    ]
    push!(tpgVar, entry)
end


println(tpgStats)
println(tpgVar)

In [ ]:
# Generate Latex file for paper
using Printf

# Latex print
if false
    file = open("../table_tpg_stats.tex", "w")

    println(
        file,
        "\t\\begin{tabular}{@{}l|$(join([@sprintf("r@{\\hspace{0.50\\tabcolsep}}l") for i in 2:7]))@{}}
\t\tGame          & \\multicolumn{2}{c}{Teams} & \\multicolumn{2}{c}{Programs} & \\multicolumn{2}{c}{Instr. / prog.} & \\multicolumn{2}{c}{Actions / game} & \\multicolumn{2}{c}{Teams / action} & \\multicolumn{2}{c}{Prog. / action} \\\\
\\hline")

    for row in 1:size(games)[1]
        print(file, "\t\t\\texttt{$(replace(tpgStats[row,1], "_" => "\\_"))} ")
        for col in 2:7
            print(file, "& $(@sprintf("%.2f", tpgStats[row,col])) & {\\scriptsize\\textcolor{gray}{\$\\pm\$ $(@sprintf("%.0f", tpgVar[row,col]))\\%}}  ")
        end
        println(file, "\\\\")
    end
    println(
        file,
        "\t\\end{tabular}"
    )
    close(file)
end

## Average variance

In [ ]:
# Run with all platforms (stack & switch)
libVariance = 100 * dbTPG[:, :tpg_time_var] ./ dbTPG[:, :tpg_time]
println("Max lib: $(maximum(libVariance)), avgLib: $(mean(libVariance))")
libVariance = 100 * dbCodegen[:, :tpg_time_var] ./ dbCodegen[:, :tpg_time]
print("Max lib: $(maximum(libVariance)), avgLib: $(mean(libVariance))")

## Speedup plot

In [ ]:
dbSpeedup = computeSpeedups(dbCodegen, dbTPG);

plotData = reduce(hcat, [dbSpeedup[dbSpeedup.game.==game, :avgSpeedup] for game in games])

avgPerPlatform = [[platforms[i], mean(plotData[i, :])] for i in 1:size(platforms)[1]]
println("Average Speedup per platform: $avgPerPlatform")

print("Speedup of asteroids / speedup of fishing derby: $(mean(plotData[:, 2] ./ plotData[:, 4]))")

groupedbar(plotData, xticks=(1:size(platforms)[1], platforms), group=repeat(games, inner=size(platforms)[1]))


In [ ]:
using PlotlyJS

# Per platform plot
# Vector with nbRunPerConfig speedups per platform. 
# reorder game for displays
speedups = [dbTPG[dbTPG.game.==game, :tpg_time] ./ dbCodegen[dbCodegen.game.==game, :tpg_time] for game in games]
p = repeat(platforms, inner=nbRunPerConfig)


# Save to pdf
b = [box(x=p, y=speedups[i], marker_size=2, name=games[i]) for i in 1:(size(games)[1])]
layout = Layout(yaxis_title="Speedup", xaxis_title="Platforms", boxmode="group", yaxis_ticksuffix="x", margin=attr(l=0, r=0, b=0, t=0), boxgap=0.2, width=1000, legend=attr(x=0.05, y=0.95,))

if false
    PlotlyJS.savefig(PlotlyJS.plot(b, layout), "../fig/speedupPerGamePerPlatform.pdf", width=500, height=350)
end

# Do display
b = [box(x=p, y=speedups[i], boxpoints="all", marker_size=2, name=games[i]) for i in 1:(size(games)[1])]
layout = Layout(yaxis_title="Inference speedup of generated code vs library", xaxis_title="Platforms", boxmode="group", yaxis_ticksuffix="x", margin=attr(l=0, r=10, b=10, t=10), boxgap=0.2, width=1000, legend=attr(x=0.05, y=0.95,))
PlotlyJS.plot(b, layout)



In [ ]:
using GLM

# Formula used for linear regression
f = @formula(Y ~ X) # alternative f = @formula(Y ~ X + X^2)

# Speedup per platform
speedupsPerPlatform = [dbTPG[dbTPG.platform.==platform, :tpg_time] ./ dbCodegen[dbCodegen.platform.==platform, :tpg_time] for platform in platforms]
p = repeat(platforms, inner=nbRunPerConfig)

# Speedup per platform : time per prog
timePerActionCodegen = DataFrame(platform=dbCodegen[:, :platform], seed=dbCodegen[:, :seed], game=dbCodegen[:, :game], time=dbCodegen[:, :tpg_time] ./ dbCodegen[:, :actions])
timePerActionTPG = DataFrame(platform=dbTPG[:, :platform], seed=dbTPG[:, :seed], game=dbTPG[:, :game], time=dbTPG[:, :tpg_time] ./ dbTPG[:, :actions])

# Get nb program executed per action
nbProgExecPerGame = dbTPGStats[:, :nbProgExec]
nbActionsPerGame = dbCodegen[1:size(games)[1]*size(seeds)[1], :actions]

nbProgExecPerAction = nbProgExecPerGame ./ nbActionsPerGame

# Get avg nb line per prog.
avgLinePerProg = dbTPGStats[:, :linePerProgram]

# Plot 1 : Speedup vs nbActions
normalizedActionExecPerGame = nbActionsPerGame ./ maximum(nbActionsPerGame)

data1 = [PlotlyJS.scatter(y=speedupsPerPlatform[i], x=normalizedActionExecPerGame, mode="markers", name="$(platforms[i])") for i in 1:size(platforms)[1]]
ols = [lm(f, DataFrame(X=normalizedActionExecPerGame, Y=speedupsPerPlatform[i])) for i in 1:size(platforms)[1]]
data1x = [[PlotlyJS.scatter(y=predict(ols[idx]), x=normalizedActionExecPerGame, mode="lines", name="$(@sprintf("%.2f",coef(ols[idx])[1])) + $(@sprintf("%.2f",coef(ols[idx])[2]))x (R²: $(@sprintf("%.2f",(r2(ols[idx])))))")] for idx in 1:size(platforms)[1]]

plot1 = PlotlyJS.plot(vcat(data1 , data1x...),
    Layout(title="Speedup = f(nbActionsPerGame)", xaxis_title="nbActions", yaxis_title="Speedups", width=600)
)

# Plot 2 : Speedup vs nbProgs
normalizedProgExecPerGame = nbProgExecPerGame ./ maximum(nbProgExecPerGame)

data2 = [PlotlyJS.scatter(y=speedupsPerPlatform[i], x=normalizedProgExecPerGame, mode="markers", name="$(platforms[i])") for i in 1:size(platforms)[1]]
ols = [lm(f, DataFrame(X=normalizedProgExecPerGame, Y=speedupsPerPlatform[i])) for i in 1:size(platforms)[1]]
data2x = [[PlotlyJS.scatter(y=predict(ols[idx]), x=normalizedProgExecPerGame, mode="lines", name="$(@sprintf("%.2f",coef(ols[idx])[1])) + $(@sprintf("%.2f",coef(ols[idx])[2]))x (R²: $(@sprintf("%.2f",(r2(ols[idx])))))")] for idx in 1:size(platforms)[1]]

plot2 = PlotlyJS.plot(vcat(data2 , data2x...),
    Layout(title="Speedup = f(NbProgPerGame)", xaxis_title="nbProgs", yaxis_title="Speedups", width=600)
)

# Plot 3 : Speedup vs nbLines per Game
normalizedLineExecPerGame = (avgLinePerProg .* nbProgExecPerGame) ./ maximum(avgLinePerProg .* nbProgExecPerGame)

data3 = [PlotlyJS.scatter(y=speedupsPerPlatform[i], x=normalizedLineExecPerGame, mode="markers", name="$(platforms[i])") for i in 1:size(platforms)[1]]

# compute errors linear
ols = [lm(f, DataFrame(X=normalizedLineExecPerGame, Y=speedupsPerPlatform[i])) for i in 1:size(platforms)[1]]
data3x = [[PlotlyJS.scatter(y=predict(ols[idx]), x=normalizedLineExecPerGame, mode="lines", name="$(@sprintf("%.2f",coef(ols[idx])[1])) + $(@sprintf("%.2f",coef(ols[idx])[2]))x (R²: $(@sprintf("%.2f",(r2(ols[idx])))))")] for idx in 1:size(platforms)[1]]
plot3 = PlotlyJS.plot(vcat(data3 , data3x...),
    Layout(title="Speedup = f(NbLinePerGame)", xaxis_title="nbLines", yaxis_title="Speedups", width=1200)
)


[plot1 plot2 plot3]


In [ ]:
# Per game plot
# Vector with nbRunPerConfig speedups per platform. 
speedups = [dbTPG[dbTPG.platform.==platform, :tpg_time] ./ dbCodegen[dbCodegen.platform.==platform, :tpg_time] for platform in platforms]
p = repeat(games, inner=nbRunPerConfig)

b = [box(x=p, y=speedups[i], boxpoints="all", marker_size=2, name=platforms[i]) for i in 1:(size(platforms)[1])]
layout = Layout(yaxis_title="Speedup", xaxis_title="Game", boxmode="group", margin=attr(l=10, r=10, b=10, t=10), boxgap=0.2, width=1000)
PlotlyJS.plot(b, layout)

## Switch vs Stack

In [ ]:
# Filter switch and stack

function computeStackVsSwitchSpeedups(dbCodegen=DataFrame, dbTPG=DataFrame)
    # Compute Avg Speed up per game per platform
    dbSpeedup = DataFrame(platform=String[],
        avgSpeedup=Float64[], medianSpeedup=Float64[], maxSpeedup=Float64[], minSpeedup=Float64[])

    platSwitch = ["xeon", "laptop", "jetson", "rpi2"]
    platStack = platSwitch .* "Stack"

    for platIndex in 1:size(platStack)[1]
        entriesStack = dbCodegen[(dbCodegen.platform.==platStack[platIndex]), :]
        entriesSwitch = dbCodegen[(dbCodegen.platform.==platSwitch[platIndex]), :]

        speedups = ((entriesStack[:, :tpg_time] ./ entriesSwitch[:, :tpg_time]) .- 1) .* 100
        speedup_stat = [platSwitch[platIndex], mean(speedups), median(speedups), maximum(speedups), minimum(speedups)]

        push!(dbSpeedup, speedup_stat)
    end
    return dbSpeedup
end

dbSpeedupStackVsSwitch = computeStackVsSwitchSpeedups(dbCodegen, dbTPG)

platSwitch = ["xeon", "laptop", "jetson", "rpi2"]
switchTime = vcat([dbCodegen[(dbCodegen.platform.==plat), :tpg_time] for plat in platSwitch]...)
stackTime = vcat([dbCodegen[(dbCodegen.platform.==plat*"Stack"), :tpg_time] for plat in platSwitch]...)

speedup = 100 * (stackTime ./ switchTime .- 1)
layout = Layout(xaxis_title="Speedup Switch vs Stack", xaxis_ticksuffix="%", margin=attr(l=0, r=0, b=0, t=0), boxgap=0.0, width=450, height=100)
b = box(x=speedup, boxpoints="all", name="", marker_size=2)

if false
    PlotlyJS.savefig(PlotlyJS.plot(b, layout), "../fig/speedupStackVsSwitch.pdf", width=450, height=100)
end

print("Average speedup: $(mean(speedup))  NbInferior:$(count(x -> (x < 0), speedup))")

PlotlyJS.plot(b, layout)


## Time per action

In [ ]:
timePerActionCodegen = DataFrame(platform=dbCodegen[:, :platform], seed=dbCodegen[:, :seed], game=dbCodegen[:, :game], time=dbCodegen[:, :tpg_time] ./ dbCodegen[:, :actions])
timePerActionTPG = DataFrame(platform=dbTPG[:, :platform], seed=dbTPG[:, :seed], game=dbTPG[:, :game], time=dbTPG[:, :tpg_time] ./ dbTPG[:, :actions])

bC = box(x=timePerActionCodegen[:, :platform], y=timePerActionCodegen[:, :time], boxpoints="all", name="Codegen", marker_size=2)
bT = box(x=timePerActionTPG[:, :platform], y=timePerActionTPG[:, :time], boxpoints="all", name="Lib", marker_size=2)
layout = Layout(yaxis_title="Time per action (log scale)", yaxis_type="log", yaxis_ticksuffix="s", xaxis_title="Platforms", boxmode="group", margin=attr(l=0, r=0, b=0, t=0), boxgap=0.2, width=1000, legend=attr(x=0.05, y=0.95))


if false
    PlotlyJS.savefig(PlotlyJS.plot([bC, bT], layout), "../fig/timePerAction.pdf", width=500, height=300)
end

PlotlyJS.plot([bC, bT], layout)


In [ ]:
# Compute Relative STD for all groups 
meanTimesCodegen = [mean(timePerActionCodegen[timePerActionCodegen.platform .== platform, :time]) for platform in platforms]
stdTimesCodegen = [std(timePerActionCodegen[timePerActionCodegen.platform .== platform, :time]) for platform in platforms]
rstdTimeCodegen = 100* stdTimesCodegen ./ meanTimesCodegen

println("## Average and relative STD for Per Action inference time." )
println(meanTimesCodegen)
println(rstdTimeCodegen)
println(mean(rstdTimeCodegen))

meanTimesLib = [mean(timePerActionTPG[timePerActionTPG.platform .== platform, :time]) for platform in platforms]
stdTimesLib = [std(timePerActionTPG[timePerActionTPG.platform .== platform, :time]) for platform in platforms]
rstdTimeLib = 100* stdTimesLib ./ meanTimesLib

println(meanTimesLib)
println(rstdTimeLib)
println(mean(rstdTimeLib))

In [ ]:
using PlotlyJS


data = [PlotlyJS.scatter(x=timePerActionTPG[timePerActionTPG.platform.==platform, :time], y=timePerActionCodegen[timePerActionCodegen.platform.==platform, :time], mode="markers", name=platform)
        for platform in platforms]


plot = hvcat((2, 2), [
        PlotlyJS.plot(
                PlotlyJS.scatter(
                        x=timePerActionTPG[timePerActionTPG.platform.==platforms[i], :time],
                        y=timePerActionCodegen[timePerActionCodegen.platform.==platforms[i], :time],
                        mode="markers", name=platforms[i],
                ),
                Layout(xaxis_ticksuffix="s", xaxis_title="Lib", yaxis_ticksuffix="s", yaxis_title="Codegen")
        )
        for i in 1:size(platforms)[1]]...
)

plot.plot.layout["width"] = 1000
plot


In [ ]:
## All in one plot
PlotlyJS.plot(data,
    Layout(xaxis_type="log", xaxis_ticksuffix="s", xaxis_title="Time Lib (log scale)", yaxis_type="log", yaxis_ticksuffix="s", yaxis_title="Time Codegen (log scale)", width=1000)
)

## Time per program

In [ ]:
# Get nb program executed per action
nbProgExecPerGame = dbTPGStats[:, :nbProgExec]
nbActionsPerGame = dbCodegen[1:size(games)[1]*size(seeds)[1], :actions]

nbProgExecPerAction = nbProgExecPerGame ./ nbActionsPerGame

# time per prog =  (time per action) / (nb prog per action)
timePerProgCodegen = DataFrame(platform=dbCodegen[:, :platform], time=timePerActionCodegen[:, :time] ./ repeat(nbProgExecPerAction, size(platforms)[1]));
timePerProgTPG = DataFrame(platform=dbCodegen[:, :platform], time=timePerActionTPG[:, :time] ./ repeat(nbProgExecPerAction, size(platforms)[1]));


In [ ]:
# Plot times per prog
bC = box(x=timePerProgCodegen[:, :platform], y=timePerProgCodegen[:, :time], boxpoints="all", name="Codegen", marker_size=2)
bT = box(x=timePerProgCodegen[:, :platform], y=timePerProgTPG[:, :time], boxpoints="all", name="Lib", marker_size=2)
layout = Layout(yaxis_title="Time per prog (log scale)", yaxis_type="log", yaxis_ticksuffix="s", xaxis_title="Platforms", boxmode="group", margin=attr(l=0, r=0, b=0, t=0), boxgap=0.2, width=1000, legend=attr(x=0.05, y=0.95))
PlotlyJS.plot([bC, bT], layout)

if false
    PlotlyJS.savefig(PlotlyJS.plot([bC, bT], layout), "../fig/timePerProgram.pdf", width=500, height=300)
end

PlotlyJS.plot([bC, bT], layout)

In [ ]:
# Compute Relative STD for all groups 
meanTimesCodegen = [mean(timePerProgCodegen[timePerProgCodegen.platform .== platform, :time]) for platform in platforms]
stdTimesCodegen = [std(timePerProgCodegen[timePerProgCodegen.platform .== platform, :time]) for platform in platforms]
rstdTimeCodegen = 100* stdTimesCodegen ./ meanTimesCodegen

println("## Average and relative STD for Per Prog inference time." )
println(meanTimesCodegen)
println(rstdTimeCodegen)
println(mean(rstdTimeCodegen))

meanTimesLib = [mean(timePerProgTPG[timePerProgTPG.platform .== platform, :time]) for platform in platforms]
stdTimesLib = [std(timePerProgTPG[timePerProgTPG.platform .== platform, :time]) for platform in platforms]
rstdTimeLib = 100* stdTimesLib ./ meanTimesLib

println(meanTimesLib)
println(rstdTimeLib)
println(mean(rstdTimeLib))

In [ ]:
using PlotlyJS


data = [PlotlyJS.scatter(x=timePerProgTPG[timePerProgTPG.platform.==platform, :time], y=timePerProgCodegen[timePerProgCodegen.platform.==platform, :time], mode="markers", name=platform)
        for platform in platforms]


plot = hvcat((2, 2), [
        PlotlyJS.plot(
                PlotlyJS.scatter(
                        x=timePerProgTPG[timePerProgTPG.platform.==platforms[i], :time],
                        y=timePerProgCodegen[timePerProgCodegen.platform.==platforms[i], :time],
                        mode="markers", name=platforms[i],
                ),
                Layout(xaxis_ticksuffix="s", xaxis_title="Lib", yaxis_ticksuffix="s", yaxis_title="Codegen")
        )
        for i in 1:size(platforms)[1]]...
)

plot.plot.layout["width"] = 1000
plot

In [ ]:
## All in one plot
PlotlyJS.plot(data,
    Layout(xaxis_type="log", xaxis_ticksuffix="s", xaxis_title="Time Lib (log scale)", yaxis_type="log", yaxis_ticksuffix="s", yaxis_title="Time Codegen (log scale)", width=1000)
)

## Time per prog line

In [ ]:
# Get avg nb line per prog.
avgLinePerProg = dbTPGStats[:, :linePerProgram]

# Time per line = (time Per Prog) / (line per prog)
# This value could be refined by extracting the number of lines of each program from the dot file, and the number of exec of each program from the raw profiling info.
timePerLineTPG = DataFrame(platform=timePerProgTPG[:, :platform], time=timePerProgTPG[:, :time] ./ repeat(avgLinePerProg, size(platforms)[1]))
timePerLineCodegen = DataFrame(platform=timePerProgCodegen[:, :platform], time=timePerProgCodegen[:, :time] ./ repeat(avgLinePerProg, size(platforms)[1]));

In [ ]:
# Plot times per line
bC = box(x=timePerLineCodegen[:, :platform], y=timePerLineCodegen[:, :time], boxpoints="all", name="Codegen", marker_size=2)
bT = box(x=timePerLineCodegen[:, :platform], y=timePerLineTPG[:, :time], boxpoints="all", name="Lib", marker_size=2)
layout = Layout(yaxis_title="Time per instruction (log scale)", yaxis_type="log", yaxis_ticksuffix="s", xaxis_title="Platforms", boxmode="group", margin=attr(l=0, r=0, b=0, t=0), boxgap=0.2, width=1000, legend=attr(x=0.05, y=0.95))

if false
    PlotlyJS.savefig(PlotlyJS.plot([bC, bT], layout), "../fig/timePerLine.pdf", width=500, height=300)
end


PlotlyJS.plot([bC, bT], layout)

In [ ]:
# Compute Relative STD for all groups 
meanTimesCodegen = [mean(timePerLineCodegen[timePerLineCodegen.platform .== platform, :time]) for platform in platforms]
stdTimesCodegen = [std(timePerLineCodegen[timePerLineCodegen.platform .== platform, :time]) for platform in platforms]
rstdTimeCodegen = 100* stdTimesCodegen ./ meanTimesCodegen

println("## Average and relative STD for Per Line inference time." )
println(meanTimesCodegen)
println(rstdTimeCodegen)
println(mean(rstdTimeCodegen))

meanTimesLib = [mean(timePerLineTPG[timePerLineTPG.platform .== platform, :time]) for platform in platforms]
stdTimesLib = [std(timePerLineTPG[timePerLineTPG.platform .== platform, :time]) for platform in platforms]
rstdTimeLib = 100* stdTimesLib ./ meanTimesLib

println(meanTimesLib)
println(rstdTimeLib)
println(mean(rstdTimeLib))

In [ ]:
using PlotlyJS


data = [PlotlyJS.scatter(x=timePerLineTPG[timePerLineTPG.platform.==platform, :time], y=timePerLineCodegen[timePerLineCodegen.platform.==platform, :time], mode="markers", name=platform)
        for platform in platforms]


plot = hvcat((2, 2), [
        PlotlyJS.plot(
                PlotlyJS.scatter(
                        x=timePerLineTPG[timePerLineTPG.platform.==platforms[i], :time],
                        y=timePerLineCodegen[timePerLineCodegen.platform.==platforms[i], :time],
                        mode="markers", name=platforms[i],
                ),
                Layout(xaxis_ticksuffix="s", xaxis_title="Lib", yaxis_ticksuffix="s", yaxis_title="Codegen")
        )
        for i in 1:size(platforms)[1]]...
)

plot.plot.layout["width"] = 1000
plot

In [ ]:
PlotlyJS.plot(data,
    Layout(xaxis_type="log", xaxis_ticksuffix="s", xaxis_title="Time Lib (log scale)", yaxis_type="log", yaxis_ticksuffix="s", yaxis_title="Time Codegen (log scale)", width=1000)
)